In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
# Import dataset from Hugging Face
from datasets import load_dataset

# Global copper supply forecasting dataset used since we'll do time-series macroeconomic forecasting later
# It makes sense to explore something similar
dataset = load_dataset("CanariaView/GlobalCopperSupplyForecastingDataset")

/Users/atulp/Documents/VSC/Django-Ticker-Prediction/macro-cycle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
